# Gallery Example: M/MAP/1 Queue (Multiclass)

This example demonstrates a multiclass M/MAP/1 queueing system:
- **Arrivals**: Exponential (Poisson) for each class
- **Service**: MAP distribution per class
- **Servers**: 1 server
- **Capacity**: Infinite
- **Scheduling**: FCFS

Each class has its own Poisson arrival stream and MAP service distribution.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mmap1_multiclass(map1=None, map2=None):
    """Create multiclass M/MAP/1 queueing model"""
    if map1 is None:
        map1 = MAP.rand(2)
        map1.set_mean(0.5)
    if map2 is None:
        map2 = MAP.rand(3)
        map2.set_mean(0.5)
    
    model = Network('M/MAP/1')
    
    # Block 1: nodes
    source = Source(model, 'mySource')
    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    # Class 1: arrival rate adjusted for 35% of total load
    oclass1 = OpenClass(model, 'myClass1')
    source.set_arrival(oclass1, Exp(0.35 / map1.get_mean()))
    queue.set_service(oclass1, map1)
    
    # Class 2: arrival rate adjusted for 15% of total load
    oclass2 = OpenClass(model, 'myClass2')
    source.set_arrival(oclass2, Exp(0.15 / map2.get_mean()))
    queue.set_service(oclass2, map2)
    
    # Block 3: topology
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_route(oclass1, queue, sink, 1.0)
    P.add_route(oclass2, source, queue, 1.0)
    P.add_route(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model

# Create the model
model = gallery_mmap1_multiclass()

## About Multiclass M/MAP/1 Queues

This model features:
- Two job classes with different characteristics
- **Class 1**: 35% of total utilization with a 2-phase MAP service
- **Class 2**: 15% of total utilization with a 3-phase MAP service

Total utilization: ρ = 0.35 + 0.15 = 0.50

The arrival rates are computed as: λᵢ = ρᵢ / E[Sᵢ]
- Where ρᵢ is the class utilization target
- E[Sᵢ] is the mean service time for class i

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)